<a href="https://colab.research.google.com/github/Pareidollya/AnaliseSentimento/blob/main/ME_1_Reviews_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalação, Configurações e Imports

In [ ]:
# Configuração do drive com as base de dados
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/BIG DATA/datasets/'

Mounted at /content/drive


In [ ]:
# Instalação do PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=1e3c69c62f8120520868b2bffd2c8f531a1349c362748e4c58c8897893acd940
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
# Instalação do Spark-NLP
!pip install spark-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.4/486.4 kB 8.6 MB/s eta 0:00:00


In [ ]:
# Imports Necessários

## Imports do Python
from collections import Counter

## Imports do PySpark
import pyspark.sql.functions as F
import pyspark.sql.types as T

## Imports do spark-nlp
import sparknlp

In [ ]:
# Instanciando o Spark com spark-nlp
spark = sparknlp.start(gpu=True)

In [ ]:
all_data = (spark
            .read
            .json(data_path + "amazon_reviews/Movies_and_TV_5.json"))

In [ ]:
# Visualizando o schema dos dados
all_data.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Format:: string (nullable = true)
 |    |-- Shape:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



#ATIVIDADE - ME 1

- **Pergunta 1:** Escolha um produto e faça a análise de como a nota variou ao longo dos anos.
- **Pergunta 2:** Cada produto é avaliado por vários usuários, sendo que a depender dessas avaliações, os valores podem sofrer variações muito altas ou não. Utilize alguma estratégia para entender os produtos que a maior e menor variação.
- **Pergunta 3:** No notebook foi feita uma análise textual mostrando, para cada produto, as 5 palavras mais citadas nos reviews. Apesar de ser uma informação simples de ser obtida, ela pode nos dar importante informações sobre o produto. No entanto, existem outras informações que podem ser obtidas. Pesquise, utilizando o `spark-nlp` como obter outras informações dos textos que nos ajude a entender melhor os produtos. Escolha pelo menos uma técnica e aplique ao conjunto de dados.


#PERGUNTA 1 - Análise por período


Apos as transformações em timestamp, é possivel fazer a filtragem por ano e listar a média do produto por ano em cada linha de uma tabela como resultado. Anos em que o produto não é avaliado também não aparecem.

In [ ]:
#transformações na data

my_data = (all_data
           .select('asin','overall','reviewText','unixReviewTime','reviewerID') # Filtra os campos de interesse
           .withColumn("reviewDate",
                       F.date_format(
                           F.from_unixtime(F.col("unixReviewTime")),"yyyy-MM-dd")) # Cria um campo data em um formato mais adequato para as análises
           .drop(F.col('unixReviewTime')) # Remove o campo unixReviewTime que foi substituído pelo campo reviewDate
)

In [ ]:
#média de avaliação para cada ano do proxuto x
(my_data.filter(F.col('asin') == '0001526863')
  .groupBy('asin', 
           F.year('reviewDate').alias('Ano'))
              .agg(F.round(F.avg('overall'), 2).alias('Avaliação'))
                .orderBy('Ano')).show()

+----------+----+---------+
|      asin| Ano|Avaliação|
+----------+----+---------+
|0001526863|2010|      5.0|
|0001526863|2013|      5.0|
|0001526863|2014|      5.0|
|0001526863|2015|      5.0|
|0001526863|2016|     4.75|
|0001526863|2017|      5.0|
+----------+----+---------+



#PERGUNTA 2 - Desvio Padrão

Através do desvio padrão é possível ver o quanto os dados estão afastados da média, concluindo que quanto maior o desvio, mais as avaliações se distanciam da média. Ilustrando caso haja um produto com menos avaliações e mais média com relação a produtos de muitas avalições. Com estes resultados será possível realizar uma analise mais aprofundada sob os dados além de sua média e numeros de avaliações.

In [ ]:
#calculando a média e desvio padrão de avaliações por produto
(my_data
      .select('asin', 'overall')
      .groupBy('asin')
        .agg(
          F.count("asin").alias("total_avaliacoes"),
          F.round(F.avg('overall'),2).alias('media'),
          F.round(F.stddev('overall'),2).alias('desvio_padrao')
)).show(20)



+----------+----------------+-----+-------------+
|      asin|total_avaliacoes|media|desvio_padrao|
+----------+----------------+-----+-------------+
|0783218923|              90| 4.51|         0.89|
|0783225911|             314| 4.68|         0.69|
|6300185117|             113| 4.65|         0.71|
|630025545X|              34| 3.94|         1.35|
|6301008944|             343| 4.62|         0.68|
|6301304977|              15|  4.2|         1.21|
|630165191X|              95| 4.27|         1.08|
|0800129016|              13| 3.85|         1.57|
|0963093932|              17| 4.71|         0.59|
|1569383529|              16| 4.63|         0.89|
|1573626163|              33|  4.7|         0.64|
|6300216144|              33| 4.48|         0.76|
|6300215628|             351| 4.57|          0.8|
|6300988597|               7| 4.71|         0.49|
|6301423682|              88|  2.9|         1.31|
|0783111509|              46| 4.61|         0.74|
|0792158202|             986| 4.72|         0.71|


#PERGUNTA 3 - ANÁLISE DE SENTIMENTO

Através de todas as informações da base é possível realizar uma análise de sentimento do texto, traçando um padrão entre os textos e suas notas é possível utilizar um modelo preditivo afim de prever qual nota o usuário daria a um texto, com base nessa informação, o objetivo será retornar se foi uma avaliação ruim, neutra ou boa.

- O modelo utilizado foi a Regressão Linear, que pelos testes entregou o melhor resultado.


Agrupamento dos textos

Nesta etapa foi selecionado os textos e suas notas, pois sao as informações necessárias para aplicar ao modelo. As notas variam numa escala de 1 a 5, porém para saber se a avaliação é ruim, neutra ou boa, so precisariamos de 3 notas, que são normalizadas para tal. Notas a baixo de 2 consideramos como "ruim" ( 1 ), notas iguais a 3 como "neutras" ( 2 ) e notas acima de 3 como "boas" ( 3 ).

> Obs: por o modelo linear tem como entrada uma coluna "label" na qual se atribui às avaliações, então foi criada uma nova coluna "label" com as notas ja normalizadas para o problema.

> Obs2: Foi necessário reduzir o tamanho do dataset, pois resultava em um tempo de fit muito alto e alguns casos atingia o limite de memória presente no collab, desta forma separamos uma pequena fração para que possa executar e fazer os testes varias vezes. Usar execução via gpu resolveria, porém o pyspark não estava usando em nenhum momento.

In [ ]:
my_data = all_data.sample(fraction = 0.03, seed = 1)

# .sample(fraction=0.01, seed=1)
text_data = (
    my_data
      .select("asin","reviewText","overall")
      .filter(F.col('reviewText').isNotNull())
)

#NORMALIZAÇÃO DAS NOTAS
text_data = text_data.withColumn('label', F.when(text_data['overall'] <= 2, 1).when(text_data['overall'] == 3, 2).otherwise(3)).drop('overall')
#verificar se há campos nulos
# text_data.filter(F.col("reviewText").isNull()).count()

In [ ]:
text_data.count()

102222

Aqui se vê necessário separar os dados para o treinamento e para o teste final, utilizando "randomSplit", é possível separa-los em 2 lista sem repetir nenhum valor em ambas.

In [ ]:
(trainingData, testData) = text_data.randomSplit([0.9, 0.01], seed=123 )
# testData.count()

In [ ]:
#imports
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.classification import LinearSVC, LogisticRegression

from sparknlp.annotator import LemmatizerModel, Stemmer, Tokenizer, StopWordsCleaner, Normalizer, YakeKeywordExtraction, Chunker, PerceptronModel
from sparknlp.base import DocumentAssembler, Finisher

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Transformações do sparkNLP para remover o máximo de caracteres indesejados e ambiguidades dos textos, pois simbolos e palavras parecidas em contextos (notas) diferentes interferem muito na precisão do modelo ao classifica-las.

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("reviewText") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("tokenized")

normalizer = Normalizer() \
    .setInputCols(["tokenized"]) \
    .setOutputCol("normalized") \
    .setLowercase(True)

lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["normalized"]) \
    .setOutputCol("lemmatized")

stop_words = StopWordsCleaner.pretrained() \
    .setInputCols(["lemmatized"]) \
    .setOutputCol("cleanTokens") \
    .setStopWords(eng_stopwords)

pos_tagger = PerceptronModel.pretrained('pos_anc') \
  .setInputCols(['document','lemmatized']) \
  .setOutputCol('pos')

allowed_tags = ['<JJ>+<NN>','<NN>+<NN>']
chunker = Chunker() \
  .setInputCols(['document','pos']) \
  .setOutputCol('ngrams') \
  .setRegexParsers(allowed_tags)

finisher = Finisher() \
  .setInputCols(['cleanTokens','ngrams'])

npl_pipeline = Pipeline(stages=[document_assembler, tokenizer, normalizer,lemmatizer,stop_words,pos_tagger, chunker, finisher])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]


Ajuste final para concatenação de duas colunas resultantes da transformação e retornar com os dados ja processados

In [ ]:
def transformData(data, label = True):
  processed = npl_pipeline.fit(data).transform(data)
  processed = processed.select('asin', 'reviewText','label', 'finished_cleanTokens','finished_ngrams' )  \
            .withColumn('finishedTokens', 
            F.concat(F.col('finished_cleanTokens'),
                      F.col('finished_ngrams')))
  return processed

In [ ]:
trainingData = transformData(trainingData)
testData = transformData(testData)

In [ ]:
trainingData.show()

+----------+--------------------+-----+--------------------+
|      asin|          reviewText|label|      finishedTokens|
+----------+--------------------+-----+--------------------+
|0005019281|Every few Christm...|    3|[every, christmas...|
|0005019281|Fine acting by He...|    3|[fine, act, henry...|
|0005019281|I have searched f...|    3|[search, film, ye...|
|0005019281|I thought his was...|    3|[think, great, tw...|
|0005019281|This is your clas...|    3|[classic, christm...|
|0005119367|This is my favori...|    3|[favorite, paul, ...|
|0005119367|         all good ty|    3| [good, ty, good ty]|
|0006486576|Slavishly faithfu...|    1|[slavishly, faith...|
|0307142493|I could never rem...|    3|[could, never, re...|
|0307142493|I love how all of...|    3|[love, rankinbass...|
|0510539610|If you leak urine...|    3|[leak, urine, mig...|
|0767001311|Don't be put off ...|    3|[dont, put, comme...|
|0767001311|The funniest Seas...|    3|[funny, season, b...|
|0767020294|I bought thi

#REGRESSÃO LINEAR - ANÁLISE DE SENTIMENTO

Regressão Linear, por ser um modelo preditivo e supervisionado ( utiliza as notas como rotulo ), é necessário realizar alguns processos a mais para que o modelo possa funcionar da melhor forma com a classificação dos textos: 

* Primeiramente é necessário criar uma representação numérica do texto, utilizando HasingTF, no qual utiliza da frequencia dos termos presente no texto, para em seguida fazer um mapeamento do mesmo como saída.

* Em seguida, a transformação IDF é usada para ajustar os pesos de cada palavra no vetor de características gerado pelo HashingTF. Ela fará o calculo da importancia de cada palavra em relação ao conjunto de documentos e ajusta os valores de cada palavra de acordo com essa importância.

* Por fim, essas transformações são uteis para melhorar o desempenho do modelo, fazendo com que ele leve em conta a importancia de cada palavra na predição do resultado. No caso da regressao com family="multinomial", as transformaçoes sao necessarias para lidar com dados textuais em formato numérico e para melhorar a precisão do modelo ao considerar a relevancia de cada palavra na classificaçao dos dados em diferentes categorias (notas).

In [ ]:
hashingTF = HashingTF(inputCol='finishedTokens', outputCol="features")
idf = IDF(inputCol='features', outputCol="tfidf")
lr = LogisticRegression(maxIter=10, regParam=0.0001, elasticNetParam=0.0, family="multinomial" )

In [ ]:
pipeline = Pipeline(stages=[hashingTF , idf, lr])

In [ ]:
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

Após todas o treinamento e finalização do modelo aplicado aos dados de teste separados anteriormente, podemos calcular sua precisão, fazendo uma contagem de quanto os dados da coluna resultante "prediction" do modelo comparados as notas reais atribuídas ao texto.

In [ ]:
def showAccuracy(predictions):
  accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(transformData(testData).count())
  # Print accuracy
  print("Accuracy = {:.2%}".format(accuracy))

In [ ]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(transformData(testData).count())

# Print accuracy
print("Accuracy = {:.2%}".format(accuracy))

Accuracy = 97.29%


sem tratamento dos dados (apenas token), sem normalização das notas
* Accuracy = 55.94% utilizando 1 % da base
* Accuracy = 58.36% utilizando 80%

com tratamentos adicionais e maior fração de dados.
* Accuracy = 67.61% 90% 90, 50, 0.01

com normalização 
* Accuracy = 83.62% 0.00005, 0.00001, maxIter=20, regParam=0.0001
* Accuracy = 83.7% 0.00005, 0.00001, maxIter=5, regParam=0.0001
* Accuracy = 93.77% com uso de ngrams 

* Accuracy = 97.29% com todos os tratamentos ( normalizer, lemmatizer, stop_words, pos_tagger, chunker, ngrams )

> execução: 15 minutos

Exibição dos resultados inserindo uma nova coluna com base em sua avaliação

In [ ]:
(predictions.select('asin','reviewText',"prediction", "label")
                      .withColumn("Avaliação", 
                          F.when(predictions.prediction == 1, "negativa")
                          .when(predictions.prediction == 2, "neutra")
                          .otherwise("boa")
                    ).show(500))

+----------+--------------------+----------+-----+---------+
|      asin|          reviewText|prediction|label|Avaliação|
+----------+--------------------+----------+-----+---------+
|0005019281|Every few Christm...|       3.0|    3|      boa|
|0005019281|Fine acting by He...|       3.0|    3|      boa|
|0005019281|I have searched f...|       3.0|    3|      boa|
|0005019281|I thought his was...|       3.0|    3|      boa|
|0005019281|This is your clas...|       3.0|    3|      boa|
|0005119367|This is my favori...|       3.0|    3|      boa|
|0005119367|         all good ty|       3.0|    3|      boa|
|0006486576|Slavishly faithfu...|       1.0|    1| negativa|
|0307142493|I could never rem...|       3.0|    3|      boa|
|0307142493|I love how all of...|       3.0|    3|      boa|
|0510539610|If you leak urine...|       3.0|    3|      boa|
|0767001311|Don't be put off ...|       3.0|    3|      boa|
|0767001311|The funniest Seas...|       3.0|    3|      boa|
|0767020294|I bought thi

Por conta da regressão linear sua precisão cai muito caso haja mais notas, pois fica cada vez mais dificil separar linearmente as categorias presentes no modelo. Normalizando para uma avaliação apenas "boa" ou "ruim" diminuiria ainda mais a margem de erro.

In [ ]:
my_data = all_data.sample(fraction = 0.03, seed = 1)
text_data = (
    my_data
      .select("asin","reviewText","overall")
      .filter(F.col('reviewText').isNotNull())
)
text_data = text_data.withColumn('label', F.when(text_data['overall'] <3, 1).otherwise(2)).drop('overall')

In [ ]:
(trainingData, testData) = text_data.randomSplit([0.9, 0.01], seed=123 )

In [ ]:
trainingData = transformData(trainingData)
testData = transformData(testData)
model = pipeline.fit(trainingData)
predictions = model.transform(testData)
showAccuracy(predictions)
predictions.select('asin','reviewText','label','prediction').show()

Accuracy = 84.65%
+----------+--------------------+-----+----------+
|      asin|          reviewText|label|prediction|
+----------+--------------------+-----+----------+
|0767809254|Steel Magnolias i...|    2|       2.0|
|0767821807|Love this movie.k...|    2|       1.0|
|0767839277|I purchased this ...|    2|       2.0|
|0767853636|Outside of "Wizar...|    2|       2.0|
|0780623614|I found this movi...|    1|       2.0|
|0782010040|john wayne at his...|    2|       2.0|
|0782010792|If you're a fan o...|    2|       2.0|
|0783110901|Very good movie t...|    2|       2.0|
|0783225857|I used this in my...|    2|       2.0|
|0783243499|American Psycho i...|    2|       1.0|
|0784011680|Love him or leave...|    2|       2.0|
|078885996X|A great take on D...|    2|       2.0|
|0790729644|Excellent product...|    2|       2.0|
|0790732181|I agree with most...|    2|       2.0|
|0790732548|Good Movie. Glad ...|    2|       2.0|
|0790734966|Interesting perso...|    2|       2.0|
|0790740443|S

Precisao real seria de 84%